In [ ]:
from google.colab import drive
drive.mount('/content/drive') 
%cd /content/drive/My\ Drive/

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
!mkdir LSB

mkdir: cannot create directory ‘LSB’: File exists


In [ ]:
%cd LSB

/content/drive/MyDrive/LSB


In [ ]:
import numpy as np
from PIL import Image

In [ ]:
def Encode(src, message, dest):

    img = Image.open(src, 'r')
    width, height = img.size
    array = np.array(list(img.getdata()))
    n = 3
    total_pixels = array.size//n

    message += "$t3g0"
    b_message = ''.join([format(ord(i), "08b") for i in message])
    req_pixels = len(b_message)

    if req_pixels > total_pixels:
        print("ERROR: Need larger file size")

    else:
        index=0
        for p in range(total_pixels):
            for q in range(0, 3):
                if index < req_pixels:
                    array[p][q] = int(bin(array[p][q])[2:9] + b_message[index], 2)
                    index += 1

        array=array.reshape(height, width, n)
        enc_img = Image.fromarray(array.astype('uint8'), img.mode)
        enc_img.save(dest)
        print("Image Encoded Successfully")

In [ ]:
def Decode(src):

    img = Image.open(src, 'r')
    array = np.array(list(img.getdata()))

    n = 3
    total_pixels = array.size//n

    hidden_bits = ""
    for p in range(total_pixels):
        for q in range(0, 3):
            hidden_bits += (bin(array[p][q])[2:][-1])

    hidden_bits = [hidden_bits[i:i+8] for i in range(0, len(hidden_bits), 8)]

    message = ""
    for i in range(len(hidden_bits)):
        if message[-5:] == "$t3g0":
            break
        else:
            message += chr(int(hidden_bits[i], 2))
    if "$t3g0" in message:
        print("Hidden Message:", message[:-5])
    else:
        print("No Hidden Message Found")

In [ ]:
def Stego():
    print("--Welcome to $t3g0--")
    print("1: Encode")
    print("2: Decode")

    func = input()

    if func == '1':
        print("Enter Source Image Path")
        src = input()
        print("Enter Message to Hide")
        message = input()
        print("Enter Destination Image Path")
        dest = input()
        print("Encoding...")
        Encode(src, message, dest)

    elif func == '2':
        print("Enter Source Image Path")
        src = input()
        print("Decoding...")
        Decode(src)

    else:
        print("ERROR: Invalid option chosen")

In [ ]:
Stego()

--Welcome to $t3g0--
1: Encode
2: Decode
1
Enter Source Image Path
/content/drive/MyDrive/LSB/input_image.png
Enter Message to Hide
Code Red, ABORT!!!
Enter Destination Image Path
/content/drive/MyDrive/LSB/output_image.png
Encoding...
Image Encoded Successfully


In [ ]:
Stego()

--Welcome to $t3g0--
1: Encode
2: Decode
2
Enter Source Image Path
/content/drive/MyDrive/LSB/output_image.png
Decoding...
Hidden Message: Code Red, ABORT!!!


In [ ]:
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim

original_image = cv2.imread("input_image.png")
steg_image = cv2.imread("output_image.png")

original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
steg_image = cv2.cvtColor(steg_image, cv2.COLOR_BGR2GRAY)

# Check if the dimensions of the images are equal
if original_image.shape != steg_image.shape:
    raise ValueError("The dimensions of the images are not equal.")

ssim_value, _ = ssim(original_image, steg_image, full=True, multichannel=False)
mse = np.mean((np.array(original_image) - np.array(steg_image)) ** 2)
max_pixel = 255
psnr = 10 * np.log10((max_pixel ** 2) / mse)

# Print the results
print(f"MSE: {mse}")
print(f"PSNR: {psnr}")
print(f"SSIM: {ssim_value}")


<ipython-input-13-2ffe53b27ccf>:15: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value, _ = ssim(original_image, steg_image, full=True, multichannel=False)


MSE: 0.0018754699519925964
PSNR: 75.3997025041281
SSIM: 0.9999885086858208


If we trained a binary classifier using the original images as positive samples and the steganographic images generated using LSB steganography as negative samples, then we can acheive a detection rate of over 99%

The presence of the hidden data in the steganographic image if we use some highly powered Steganalysis tools like StegExpose